In [10]:
import pandas as pd
import numpy as np
from binance.client import Client
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None 

In [11]:
class Strategy():
    def __init__(self, client, symbols, timeframe, startdate = "2 year ago UTC", service_charge = 0.00075, money = 1000, storage = 0):
        self.client = client
        self.symbols = symbols
        self.timeframe = timeframe
        self.startdate = startdate
        self.service_charge = service_charge
        self.start_money = [money] * len(symbols)
        self.start_storage = [storage] * len(symbols)

        self.all_data = []
        for coin in symbols:
            bars = client.get_historical_klines(symbol=f'{coin}USDT',interval=timeframe,start_str=self.startdate)
            test_df = pd.DataFrame(bars[:],columns=["timestamp","open","high","low","close","volume", "close_time", "quote_asset_volume", "number_of_trade", "TBB", "TBQ", "ignore"])
            test_df["date"]=pd.to_datetime(test_df["timestamp"],unit="ms").astype(str)
            test_df = test_df.drop(["timestamp", "close_time", "quote_asset_volume", "number_of_trade", "TBB", "TBQ", "ignore"], axis=1)
            test_df["open"] = pd.to_numeric(test_df["open"])
            test_df["high"] = pd.to_numeric(test_df["high"])
            test_df["low"] = pd.to_numeric(test_df["low"])
            test_df["close"] = pd.to_numeric(test_df["close"])
            test_df["volume"] = pd.to_numeric(test_df["volume"])
            test_df["signal"] = ["None"] * len(test_df)
            self.all_data.append(test_df)

        self.profit = [0] * len(symbols)
    def print(self):
        print("Time frame", self.timeframe)
        print("Start from", self.startdate)

        for i in range(len(self.symbols)):
            print("=============")
            print(self.symbols[i], self.profit[i])

        print("=============")
        print("average", sum(self.profit) / len(self.profit))

    def decide_action(self):
        pass

    def back_test(self, type = "all in", fixed_money = None):
        for i in range(len(self.symbols)):
            money = self.start_money[i]
            storage = self.start_storage[i]
            for j in range(len(self.all_data[i])):
                if (self.all_data[i]["signal"][j] == "buy"):
                    if (type == "all in"):
                        if (storage < 0):
                            money -= -1 * storage * self.all_data[i]["close"][j]
                            storage = 0
                        storage += money / self.all_data[i]["close"][j] 
                        money = 0
                    else:
                        if (storage > 0):
                            money -= -1 * storage * self.all_data[i]["close"][j]
                            storage = 0
                        storage += fixed_money[i] / self.all_data[i]["close"][j] 
                        money -= fixed_money[i]
                elif (self.all_data[i]["signal"][j] == "sell"):
                    if (type == "all in"):
                        if (storage > 0):
                            money += storage * self.all_data[i]["close"][j]
                            storage = 0
                        storage -= money / self.all_data[i]["close"][j] 
                        money += money
                    else:
                        if (storage > 0):
                            money += storage * self.all_data[i]["close"][j]
                            storage = 0
                        storage -= fixed_money[i] / self.all_data[i]["close"][j] 
                        money += fixed_money[i]
            self.profit[i] = (money + storage * self.all_data[i]["close"][len(self.all_data[i]) - 1]) / self.start_money[i]

In [12]:
class EMA(Strategy):
    def decide_action(self, smooth = 2, short_period = 10, long_period = 100, stable_offset = 150):
        for i in range(len(self.all_data)):
            self.all_data[i]["short EMA"] = self.all_data[i]["close"].copy()
            self.all_data[i]["long EMA"] = self.all_data[i]["close"].copy()
            self.all_data[i]["long SMA"] = self.all_data[i]["close"].copy()

            for j in range(1, len(self.all_data[i])):
                self.all_data[i]["short EMA"][j] = self.all_data[i]["short EMA"][j] * smooth / (short_period + 1) + self.all_data[i]["short EMA"][j - 1] * (1 - smooth / (short_period + 1))
                self.all_data[i]["long EMA"][j] = self.all_data[i]["long EMA"][j] * smooth / (long_period + 1) + self.all_data[i]["long EMA"][j - 1] * (1 - smooth / (long_period + 1))
            # for j in range(long_period, len(self.all_data[i])):
            #     self.all_data[i]["long SMA"][j] = self.all_data[i]["close"][j - long_period:j].mean()
            for j in range(stable_offset, len(self.all_data[i])):
                if (self.all_data[i]["short EMA"][j - 1] > self.all_data[i]["long EMA"][j - 1] and self.all_data[i]["short EMA"][j - 2] < self.all_data[i]["long EMA"][j - 2]):
                    self.all_data[i]["signal"][j] = "buy"
                # if (self.all_data[i]["short EMA"][j - 1] > self.all_data[i]["long SMA"][j - 1] and self.all_data[i]["short EMA"][j - 2] < self.all_data[i]["long SMA"][j - 2]):
                #     self.all_data[i]["signal"][j] = "buy"
                elif (self.all_data[i]["short EMA"][j - 1] < self.all_data[i]["long EMA"][j - 1] and self.all_data[i]["short EMA"][j - 2] > self.all_data[i]["long EMA"][j - 2]):
                    self.all_data[i]["signal"][j] = "sell"


In [13]:
profits = []
count = 0
target_symbol = ["ETH", "BTC"]
time_frames = ["1d", "12h", "6h", "4h", "2h", "1h"]
start_str = "1 month ago UTC"
client = Client("YjY0w2BJZZ35o2Q9So1OvRP1gm3exc01LKp2yyLpMth5xmqVn0tQosRYOPrnb7la", "5rRXJ2rOa96hVPZTTOfVP9H4blP6AMt2QN2M0RWX2cMq9PngbJ5MEK88eGB9I4YD")
res = []
for timeframe in time_frames:
    for short_period in range(10, 20):
        for long_period in range(100,130):
            a = EMA(client, target_symbol, timeframe, start_str)
            b = EMA(client, target_symbol, timeframe, start_str)
            a.decide_action(short_period=short_period, long_period=long_period)
            break
        break
    break
            # a.decide_action(short_period=short_period, long_period=long_period)
            # b.decide_action(short_period=short_period, long_period=long_period)
            # a.back_test(type="fixed_money", fixed_money=[1000] * len(target_symbol))
            # b.back_test(type="all in", fixed_money=[1000] * len(target_symbol))
            # res.append([short_period, long_period, timeframe, sum(a.profit) / len(a.profit)])
            # print("short", short_period, "long", long_period, "timeframe", timeframe, "average","fixed money", sum(a.profit) / len(a.profit), "all in", sum(b.profit) / len(b.profit))

In [14]:
a.all_data[0]

,open,high,low,close,volume,date,signal,short EMA,long EMA,long SMA
0,1144.05,1156.80,1087.42,1100.21,1.295518e+06,2022-06-29,None,1100.210000,1100.210000,1100.21
1,1100.21,1106.62,998.00,1071.01,1.420670e+06,2022-06-30,None,1094.900909,1099.631782,1071.01
2,1071.02,1117.00,1033.44,1059.73,1.562255e+06,2022-07-01,None,1088.506198,1098.841648,1059.73
3,1059.73,1078.88,1028.29,1067.01,7.148121e+05,2022-07-02,None,1084.597799,1098.211318,1067.01
4,1067.01,1088.34,1040.78,1074.26,6.447463e+05,2022-07-03,None,1082.718199,1097.737035,1074.26
5,1074.26,1160.90,1045.37,1151.00,1.150109e+06,2022-07-04,None,1095.133072,1098.791747,1151.00
6,1151.01,1174.90,1076.34,1132.50,1.398961e+06,2022-07-05,None,1101.927059,1099.459237,1132.50
7,1132.51,1205.00,1111.44,1186.57,1.211768e+06,2022-07-06,None,1117.316684,1101.184203,1186.57
8,1186.57,1254.29,1163.18,1237.49,1.001316e+06,2022-07-07,None,1139.166378,1103.883327,1237.49
9,1237.49,1276.46,1193.15,1214.04,1.189518e+06,2022-07-08,None,1152.779764,1106.064648,1214.04
